In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('CUDA version:', torch.version.cuda)
    print('Memory Usage:') 
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce GTX 960
CUDA version: 11.7
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
import numpy as np
import PIL
from PIL import Image
import datasets
import evaluate
import torch
import json
import codecs
import os
from os import sys

from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation, TrainingArguments, Trainer, EarlyStoppingCallback

/home/kate/miniconda3/envs/master/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
module_path = os.path.abspath(os.path.join('./src'))
if module_path not in sys.path:
    sys.path.append(module_path)
from data_prepossessing import create_datasets_for_plants, get_labels
from constants import *

In [6]:
checkpoint = "nvidia/mit-b0"
image_processor = SegformerImageProcessor.from_pretrained(checkpoint)
image_processor

/home/kate/miniconda3/envs/master/lib/python3.11/site-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


SegformerImageProcessor {
  "do_normalize": true,
  "do_reduce_labels": false,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "SegformerFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "SegformerImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 512,
    "width": 512
  }
}

In [7]:
def train_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels, return_tensors="pt")
    return inputs

In [8]:
metric = evaluate.load("mean_iou")

In [9]:
def compute_metrics(num_labels, eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = torch.nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [10]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=50,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    eval_steps=100,
    logging_steps=1,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    seed=seed,
)

In [11]:
def initialize_trainer(model, num_labels, train_ds, test_ds) :
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=lambda eval_pred: compute_metrics(num_labels, eval_pred),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    return trainer

In [12]:
def model_init():
	model_type = "multiclass"
	crop = "broad_bean" 
	id2label, label2id = get_labels(crop, model_type)
	return AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

In [13]:
def sigopt_hp_space(trial):
    return [
        {"bounds": {"min": 1e-6, "max": 1e-4}, "name": "learning_rate", "type": "double"}
    ]

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
    }

In [16]:
import optuna
optuna.__version__

'3.3.0'

In [14]:
def initialize_trainer_for_hp_search(num_labels, train_ds, test_ds) :
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=lambda eval_pred: compute_metrics(num_labels, eval_pred),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    best_trial = trainer.hyperparameter_search(
        direction="minimize",
        backend="sigopt",
        hp_space=optuna_hp_space,
        n_trials=20,
    )

    return best_trial

In [15]:
model_type = "multiclass"
crop = "broad_bean" 
model_plant_names = [crop] + weed_plants
train_ds, val_ds, test_ds = create_datasets_for_plants(model_plant_names, model_type, crop)

train_ds.set_transform(train_transforms)
val_ds.set_transform(train_transforms)
test_ds.set_transform(train_transforms)

id2label, label2id = get_labels(crop, model_type)

# model = model_init(id2label, label2id)
best_trial = initialize_trainer_for_hp_search(len(id2label), train_ds, val_ds)
best_trial
# trainer.train()

['img_00173.png', 'img_00174.png', 'img_00175.png', 'img_00176.png', 'img_00177.png', 'img_00178.png', 'img_00672.png', 'img_00673.png', 'img_00674.png', 'img_00675.png', 'img_00676.png', 'img_00677.png', 'img_00678.png', 'img_00679.png', 'img_00680.png', 'img_00681.png', 'img_00682.png', 'img_00683.png', 'img_00684.png', 'img_00882.png', 'img_00883.png', 'img_00884.png', 'img_00885.png', 'img_00886.png', 'img_00887.png', 'img_00938.png', 'img_00980.png', 'img_00981.png', 'img_00982.png', 'img_00983.png', 'img_00984.png', 'img_00985.png', 'img_00986.png', 'img_00987.png', 'img_00988.png', 'img_00989.png', 'img_01070.png', 'img_01071.png', 'img_01072.png', 'img_01073.png', 'img_01074.png', 'img_01075.png', 'img_01076.png', 'img_01077.png', 'img_01078.png', 'img_01079.png', 'img_01219.png', 'img_01220.png', 'img_01221.png', 'img_01222.png', 'img_01223.png', 'img_01224.png', 'img_01225.png', 'img_01226.png', 'img_01227.png', 'img_01228.png', 'img_01279.png', 'img_01280.png', 'img_01281.pn

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.2.proj.weight', 'decode_head.classifier.weight', 'decode_head.linear_fuse.weight', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.bias', 'decode_head.batch_norm.weight', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.2.proj.bias', 'decode_head.classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'NoneType' object has no attribute 'suggest_float'

In [13]:
def train_model_of_type_for_crop(model_type, crop):
    model_plant_names = [crop] + weed_plants
    train_ds, val_ds, test_ds = create_datasets_for_plants(model_plant_names, model_type, crop)

    print("Training subset number of images: " + str(train_ds.num_rows))
    print("Validation subset number of images: " + str(val_ds.num_rows))
    print("Test subset number of images: " + str(test_ds.num_rows))

    train_ds.set_transform(train_transforms)
    val_ds.set_transform(train_transforms)
    test_ds.set_transform(train_transforms)

    id2label, label2id = get_labels(crop, model_type)

    print('Number of classes:', len(id2label))
    print('id2label:', id2label)
    print('label2id:', label2id)

    model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)
    trainer = initialize_trainer(model, len(id2label), train_ds, val_ds)
    trainer.train()

    # Save the trained model, so that it can be used for inference later.
    # Save the log history, so that it can be used for plotting later.
    trainer.save_model('models/' + model_type + '/' + crop)
    with open('models/' + model_type + '/' + crop + '/log_history.json', 'w') as file:
        log_history = trainer.state.log_history
        json.dump(log_history, file)

    test_metric = trainer.evaluate(test_ds)
    test_metric

    with open('models/' + model_type + '/' + crop + '/test_metric.json', 'w') as file:
        json.dump(test_metric, file)

In [14]:
train_model_of_type_for_crop("multiclass", "broad_bean")
# train_model_of_type_for_crop("multiclass", "sugar_beet")

['img_00071.png', 'img_00072.png', 'img_00074.png', 'img_00083.png', 'img_00084.png', 'img_00085.png', 'img_00151.png', 'img_00152.png', 'img_00153.png', 'img_00154.png', 'img_00155.png', 'img_00156.png', 'img_00210.png', 'img_00211.png', 'img_00212.png', 'img_00213.png', 'img_00214.png', 'img_00215.png', 'img_00216.png', 'img_00217.png', 'img_00218.png', 'img_00220.png', 'img_00221.png', 'img_00226.png', 'img_00227.png', 'img_00229.png', 'img_00248.png', 'img_00249.png', 'img_00250.png', 'img_00251.png', 'img_00254.png', 'img_00255.png', 'img_00256.png', 'img_00257.png', 'img_00258.png', 'img_00259.png', 'img_00260.png', 'img_00264.png', 'img_00265.png', 'img_00266.png', 'img_00267.png', 'img_00268.png', 'img_00269.png', 'img_00271.png', 'img_00293.png', 'img_00294.png', 'img_00295.png', 'img_00296.png', 'img_00297.png', 'img_00298.png', 'img_00299.png', 'img_00300.png', 'img_00301.png', 'img_00302.png', 'img_00303.png', 'img_00379.png', 'img_00380.png', 'img_00381.png', 'img_00382.pn

['img_01804.png', 'img_01807.png', 'img_01790.png', 'img_01803.png', 'img_01793.png', 'img_01797.png', 'img_01798.png', 'img_01796.png', 'img_01799.png', 'img_01795.png', 'img_01791.png', 'img_01802.png', 'img_01806.png', 'img_01789.png', 'img_01805.png', 'img_01800.png', 'img_01794.png', 'img_01801.png', 'img_01792.png', 'img_01788.png']
Number of plant images for plant corn_spurry : 20
['img_01768.png', 'img_01767.png', 'img_01796.png']
Number of plant images for plant red-root_amaranth : 3
['img_01812.png', 'img_01809.png', 'img_01815.png', 'img_01810.png', 'img_01808.png', 'img_01813.png', 'img_01814.png', 'img_01811.png']
Number of plant images for plant red_fingergrass : 8
['img_00434.png', 'img_00203.png', 'img_00424.png', 'img_00717.png', 'img_00657.png', 'img_00430.png', 'img_00715.png', 'img_00420.png', 'img_00658.png', 'img_00209.png', 'img_00659.png', 'img_00419.png', 'img_00709.png', 'img_00713.png', 'img_00206.png', 'img_00429.png', 'img_00712.png', 'img_00421.png', 'img_

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.linear_c.2.proj.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_var', 'decode_head.classifier.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.classifier.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.batch_norm.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.batch_norm.running_mean', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.3.proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: To use hyperparameter search, you need to pass your model through a model_init function.

In [ ]:
# import subprocess
# from typing import NoReturn

# def shutdown_windows() -> NoReturn:
#     subprocess.run(["shutdown", "/s", "/t", "0"])

# shutdown_windows()